In [1]:
import os
from typing import List
from pydantic import BaseModel, Field
from pprint import pprint
import instructor

from dotenv import load_dotenv
from openai import OpenAI

# Load env variables
load_dotenv()

# Define clients
groq = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

In [2]:
# The task will be to extract a list of names from the given text
text = """John Smith is a software engineer who works with Sarah Johnson and Mike Brown. 
They often collaborate with Emily Chen, David Martinez, and Lisa Wong on various projects.
"""

In [3]:
# 1. Define the model
class NameExtractor(BaseModel):
    names: List[str]

patched_client = instructor.from_openai(groq)

structured_response = patched_client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "user", "content": text}
    ],
    response_model=NameExtractor
)

In [4]:
structured_response.model_dump()

{'names': ['John Smith',
  'Sarah Johnson',
  'Mike Brown',
  'Emily Chen',
  'David Martinez',
  'Lisa Wong']}

In [5]:
system_message = "your task is to select the tool based on the user query. YOU ONLY DECIDE ABOUT THE TOOL TO USE. You can select the following: ANSWER, to directly answer the question and then show the answer as the value or SEARCH, to perform web search, and then show keywords"
user_message = "What is the weather today?"

class ResponseModel(BaseModel):
    tool: str = Field(description="The tool to use")
    value: str = Field(description="The variables to use")

structured_response = patched_client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ],
    response_model=ResponseModel
)

output_variables = structured_response.model_dump()

pprint(output_variables)

{'tool': 'SEARCH', 'value': 'weather today'}


In [6]:
def web_search(keywords: str):
    # Perform a web search
    print(f"Performing a web search for {keywords}...")

tool = output_variables["tool"]

if tool == "SEARCH":
    web_search(output_variables["value"])

Performing a web search for weather today...
